# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("./output_data/cities.csv", encoding="utf-8")
weather_df.head()

,city_name,longitude,latitude,max_temp,humidity,cloudiness,wind_speed,country,city_date
0,Rikitea,-134.97,-23.12,76.68,69,93,9.86,PF,1586852334
1,Ambon City,128.18,-3.70,77.00,94,40,19.46,ID,1586852334
2,Saldanha,17.94,-33.01,66.20,69,0,11.41,ZA,1586852334
3,Hailey,-114.32,43.52,28.40,46,1,9.17,US,1586852335
4,San Patricio,-104.70,19.22,71.10,50,0,4.83,MX,1586852335


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_df[["latitude", "longitude"]].astype(float)
humidity = weather_df.humidity
fig = gmaps.figure(zoom_level = 2, center=(41.37, -73.41), layout={
        'width': '1000px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=2 )
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='1000px'))

--> According to humidity heatmap, we may confirm humidity level is higher at sea shore. And humidity and latitude has no corelation.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
# A max temperature lower than 80 degrees but higher than 70.
vacation_place_df = weather_df.loc[(weather_df.max_temp >= 70)&(weather_df.max_temp <90)]
# humidity less than 60% but higher than 20
vacation_place_df = vacation_place_df.loc[(vacation_place_df.humidity <=60) & (vacation_place_df.humidity >20)]
# Wind speed less than 10 mph.
# cloudiness less than 50 %
vacation_place_df = vacation_place_df.loc[(vacation_place_df.wind_speed <=10) & (vacation_place_df.cloudiness < 50)]
hotel_df = vacation_place_df.dropna()
hotel_df.reset_index(drop=True, inplace=True)
hotel_df
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

,city_name,longitude,latitude,max_temp,humidity,cloudiness,wind_speed,country,city_date
0,San Patricio,-104.70,19.22,71.10,50,0,4.83,MX,1586852335
1,Hong Kong,114.16,22.29,75.99,53,20,6.93,HK,1586852083
2,Ambovombe,46.08,-25.17,88.03,42,0,7.34,MG,1586852362
3,Bushehr,50.84,28.97,77.00,47,0,9.17,IR,1586852362
4,Kiryat Gat,34.76,31.61,73.00,56,0,6.93,IL,1586852147
5,Mae Chan,99.85,20.15,88.00,58,20,4.70,TH,1586852383
6,Solwezi,26.39,-12.17,75.49,49,36,4.59,ZM,1586852384
7,Ambanja,48.45,-13.68,86.00,57,40,4.70,MG,1586852384
8,Xichang,102.26,27.90,72.00,39,0,5.23,CN,1586852385
9,Coahuayana Viejo,-103.68,18.73,71.19,41,8,3.13,MX,1586852385


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
data_result =[]
hotel_df["Hotel Name"] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = str(row["latitude"])
    lng = str(row["longitude"])
    query_url = f"{base_url}?key={g_key}&location={lat},{lng}&radius=5000&type=lodging&keyword=hotel"
    data = requests.get(query_url)
    data_json = data.json()
    try:
        hotel_result = data_json["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_result
        print(f"{index}, hotel info is {hotel_result}")
    except:
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("hotel info is not found.")


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0, hotel info is Hotel Trivento
1, hotel info is Dorsett Mongkok, Hong Kong
2, hotel info is Hotel Le Source
3, hotel info is Delvar Hotel
4, hotel info is Desert Gat
5, hotel info is Rimchan Resort Hotel
6, hotel info is Royal Solwezi Hotel
hotel info is not found.
8, hotel info is Qionghai Bay Bonreal Hotel
9, hotel info is Hotel Los Arcos
10, hotel info is Hotel Ritz Lauca
11, hotel info is Hotel Mazino
12, hotel info is Chadiza Guest House
13, hotel info is Princesse Tsiribihina
14, hotel info is Alfresco Beach Hotel
15, hotel info is Agorà Palace Hotel
16, hotel info is Chikho Hotel
17, hotel info is Royal Hotel Moree
18, hotel info is Blackwater Central Motel
19, hotel info is Pullman Zhuhai
hotel info is not found.
21, hotel info is Hotel Poufiret
22, hotel info is Bays Garden Hotel


In [7]:
hotel_df

,city_name,longitude,latitude,max_temp,humidity,cloudiness,wind_speed,country,city_date,Hotel Name
0,San Patricio,-104.70,19.22,71.10,50,0,4.83,MX,1586852335,Hotel Trivento
1,Hong Kong,114.16,22.29,75.99,53,20,6.93,HK,1586852083,"Dorsett Mongkok, Hong Kong"
2,Ambovombe,46.08,-25.17,88.03,42,0,7.34,MG,1586852362,Hotel Le Source
3,Bushehr,50.84,28.97,77.00,47,0,9.17,IR,1586852362,Delvar Hotel
4,Kiryat Gat,34.76,31.61,73.00,56,0,6.93,IL,1586852147,Desert Gat
5,Mae Chan,99.85,20.15,88.00,58,20,4.70,TH,1586852383,Rimchan Resort Hotel
6,Solwezi,26.39,-12.17,75.49,49,36,4.59,ZM,1586852384,Royal Solwezi Hotel
7,Ambanja,48.45,-13.68,86.00,57,40,4.70,MG,1586852384,NaN
8,Xichang,102.26,27.90,72.00,39,0,5.23,CN,1586852385,Qionghai Bay Bonreal Hotel
9,Coahuayana Viejo,-103.68,18.73,71.19,41,8,3.13,MX,1586852385,Hotel Los Arcos


In [8]:
cleaned_hotel_df = hotel_df.dropna()
cleaned_hotel_df.reset_index(inplace=True, drop=True)
cleaned_hotel_df

,city_name,longitude,latitude,max_temp,humidity,cloudiness,wind_speed,country,city_date,Hotel Name
0,San Patricio,-104.70,19.22,71.10,50,0,4.83,MX,1586852335,Hotel Trivento
1,Hong Kong,114.16,22.29,75.99,53,20,6.93,HK,1586852083,"Dorsett Mongkok, Hong Kong"
2,Ambovombe,46.08,-25.17,88.03,42,0,7.34,MG,1586852362,Hotel Le Source
3,Bushehr,50.84,28.97,77.00,47,0,9.17,IR,1586852362,Delvar Hotel
4,Kiryat Gat,34.76,31.61,73.00,56,0,6.93,IL,1586852147,Desert Gat
5,Mae Chan,99.85,20.15,88.00,58,20,4.70,TH,1586852383,Rimchan Resort Hotel
6,Solwezi,26.39,-12.17,75.49,49,36,4.59,ZM,1586852384,Royal Solwezi Hotel
7,Xichang,102.26,27.90,72.00,39,0,5.23,CN,1586852385,Qionghai Bay Bonreal Hotel
8,Coahuayana Viejo,-103.68,18.73,71.19,41,8,3.13,MX,1586852385,Hotel Los Arcos
9,Menongue,17.69,-14.66,76.89,41,5,5.35,AO,1586852393,Hotel Ritz Lauca


In [9]:
city_locations = weather_df[["latitude", "longitude"]].astype(float)
humidity = weather_df.humidity

pref_city_locations = cleaned_hotel_df[["latitude", "longitude"]].astype(float)

fig = gmaps.figure(zoom_level = 2, center=(41.37, -73.41), layout={
        'width': '1000px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
heat_layer = gmaps.heatmap_layer(city_locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=1 )
marker_layer = gmaps.marker_layer(pref_city_locations, hover_text=cleaned_hotel_df["city_name"], info_box_content=cleaned_hotel_df["Hotel Name"] )
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='1000px'))

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]
locations = cleaned_hotel_df[["latitude", "longitude"]]

In [11]:
# Add marker layer ontop of heat map

humidity = cleaned_hotel_df.humidity

fig = gmaps.figure(zoom_level = 2, center=(41.37, -73.41), layout={
        'width': '1000px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3 )
marker_layer = gmaps.marker_layer(locations, hover_text=cleaned_hotel_df["city_name"], info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='1000px'))

The city at seaside has more favorable weather for vacation. and every cities are within latitude range [-30, 45] because of temperature.